In [ ]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import time

In [ ]:
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

# Séparation des images en ensembles d'entraînement, validation et test

In [ ]:
import os, shutil 

original_dataset_dir= 'D:/Documents/SYS866_Projet_Final/Grad-CAM/HCP_SYS866_2020/'

base_dir='D:/Documents/SYS866_Projet_Final/Grad-CAM/cerveaux_resnet/'
if not os.path.exists(base_dir):
    os.mkdir(base_dir)
    
train_dir=os.path.join(base_dir,'train/')
if not os.path.exists(train_dir):
    os.mkdir(train_dir)
    
validation_dir=os.path.join(base_dir,'validation/')
if not os.path.exists(validation_dir):
    os.mkdir(validation_dir)
    
test_dir=os.path.join(base_dir,'test/')
if not os.path.exists(test_dir):
    os.mkdir(test_dir)

In [ ]:
train_homme_dir=os.path.join(train_dir,'homme')
if not os.path.exists(train_homme_dir):
    os.mkdir(train_homme_dir)

train_femme_dir = os.path.join(train_dir, 'femme')
if not os.path.exists(train_femme_dir):
    os.mkdir(train_femme_dir)
    
validation_homme_dir=os.path.join(validation_dir,'homme')
if not os.path.exists(validation_homme_dir):
    os.mkdir(validation_homme_dir)

validation_femme_dir = os.path.join(validation_dir, 'femme')
if not os.path.exists(validation_femme_dir):
    os.mkdir(validation_femme_dir)
    
test_homme_dir=os.path.join(test_dir,'homme')
if not os.path.exists(test_homme_dir):
    os.mkdir(test_homme_dir)

test_femme_dir = os.path.join(test_dir, 'femme')
if not os.path.exists(test_femme_dir):
    os.mkdir(test_femme_dir)

In [ ]:
file= open('D:/Documents/SYS866_Projet_Final/Grad-CAM/labels.csv','r')

ligne= file.readlines()

file.close()

In [ ]:
nom=[]
sexe=[]


for i in range (2,len(ligne)):
    
    li=str(ligne[i]).rstrip('\n')
    liste=li.split(',')
    nom.append(liste[0])
    sexe.append(liste[8])
    
print(len(nom))
print(len(sexe))

In [ ]:
# Ajout d'images au dossier d'entrainement
for i in range(0, 850):
    fname='img'+nom[i]+'_T1w.png'
    src= os.path.join(original_dataset_dir, fname)
    if os.path.isfile(src)==True:
        if sexe[i]=='1':
            src= os.path.join(original_dataset_dir, fname)
            dst=os.path.join(train_homme_dir, fname)
            shutil.copyfile(src, dst)
        else:
            src= os.path.join(original_dataset_dir, fname)
            dst=os.path.join(train_femme_dir, fname)
            shutil.copyfile(src, dst)

In [ ]:
# Ajout d'images au dossier de validation
for i in range(850, 1050):
    fname='img'+nom[i]+'_T1w.png'
    src= os.path.join(original_dataset_dir, fname)
    if os.path.isfile(src)==True:
        if sexe[i]=='1':
            src= os.path.join(original_dataset_dir, fname)
            dst=os.path.join(validation_homme_dir, fname)
            shutil.copyfile(src, dst)
        else:
            src= os.path.join(original_dataset_dir, fname)
            dst=os.path.join(validation_femme_dir, fname)
            shutil.copyfile(src, dst)

In [ ]:
# Ajout d'images au dossier de test
for i in range(1050, 1205):
    fname='img'+nom[i]+'_T1w.png'
    src= os.path.join(original_dataset_dir, fname)
    if os.path.isfile(src)==True:
        if sexe[i]=='1':
            src= os.path.join(original_dataset_dir, fname)
            dst=os.path.join(test_homme_dir, fname)
            shutil.copyfile(src, dst)
        else:
            src= os.path.join(original_dataset_dir, fname)
            dst=os.path.join(test_femme_dir, fname)
            shutil.copyfile(src, dst)

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        rotation_range=5)

validation_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

In [ ]:
image_size = (150, 180)
batch_size = 16

training_set = train_datagen.flow_from_directory(
    "cerveaux_resnet/train",
    target_size=(image_size),
    batch_size=batch_size,
    class_mode='categorical'
)

In [ ]:
validation_set = validation_datagen.flow_from_directory(
    "cerveaux_resnet/validation",
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical'
)

In [ ]:
test_datagen = ImageDataGenerator(rescale=1./255)

test_set = test_datagen.flow_from_directory(
    "cerveaux_resnet/test",
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical',
    shuffle = False
)

## Définition du modèle

In [ ]:
densenet = tf.keras.applications.DenseNet201(
    include_top=False,
    weights="imagenet",
    input_tensor=None,
    input_shape=(150,150,3),
    pooling=None,
    classes=1000,
)

In [ ]:
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model

# add a global spatial average pooling layer
x = densenet.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(2, activation='softmax')(x)

In [ ]:
# this is the model we will train
densenet_model = Model(inputs=densenet.input, outputs=predictions)

In [ ]:
# first: train only the top layers (which were randomly initialized)
# i.e. freeze all convolutional Densenet layers
for layer in densenet.layers:
    layer.trainable = False

# Entraînement du modèle

In [ ]:
densenet_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
start_time = time.time()

history = densenet_model.fit(x = training_set, validation_data=validation_set, shuffle=True, epochs=21)

print(time.time() - start_time)

# Séparation des images en ensembles d'entraînement, validation et test

In [ ]:
import matplotlib.pyplot as plt

acc= history.history['accuracy']
val_acc = history.history['val_accuracy']
loss= history.history['loss']
val_loss= history.history['val_loss']

epochs= range(1, len(acc)+1)

plt.plot(epochs, acc, 'bo', label='entrainement')
plt.plot(epochs, val_acc,'b', label='validation')
plt.title('Exactitude pendant l\'entrainement et la validation')
plt.legend()

plt.figure()
plt.plot(epochs, loss, 'bo', label='entrainement')
plt.plot(epochs, val_loss, 'b', label='validation')
plt.title('Perte pendant l\'entrainement et la validation')
plt.legend()
plt.show()

In [ ]:
test_loss, test_acc= densenet_model.evaluate(test_set)

In [ ]:
y_true_labels = test_set.classes
true_labels = []

for label in y_true_labels:
    true_labels.append(np.array([np.float32(label)]))

In [ ]:
preds = densenet_model.predict(test_set)

In [ ]:
from sklearn.metrics import roc_curve, auc

In [ ]:
predictions = []

for x in preds.tolist():
    predictions.append(x[1])

In [ ]:
fpr, tpr, _ = roc_curve(true_labels, predictions)
auc = auc(fpr, tpr)

In [ ]:
plt.figure(1)
plt.plot([0, 1], [0, 1], 'k--')
plt.plot(fpr, tpr, label='AUC = {:.3f}'.format(auc))
plt.xlabel('False positive rate')
plt.ylabel('True positive rate')
plt.title('ROC curve')
plt.legend(loc='best')
plt.show()

## Enregistrement du modèle

In [ ]:
densenet_model.save('D:/Documents/SYS866_Projet_Final/Grad-CAM/models/densenet_adam__categorical')

### Chargement du modèle

In [ ]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt

densenet_model = keras.models.load_model('D:/Documents/SYS866_Projet_Final/Grad-CAM/models/densenet_adam__categorical')